# IESB - CIA035 - Aula 12 - Redes Neurais
### Primeira rede com PyTorch

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


In [2]:
# Carregando o dataset
df = pd.read_csv('/kaggle/input/iris/Iris.csv')

df.shape

(150, 6)

In [3]:
# Verificando os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [4]:
# Olhando os dados
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
# Olhando a coluna Species
df['Species'].value_counts()

Iris-virginica     50
Iris-versicolor    50
Iris-setosa        50
Name: Species, dtype: int64

In [6]:
# Convertendo a coluna Species
df['Species'] = df['Species'].map({'Iris-versicolor': 0, 'Iris-setosa': 1, 'Iris-virginica': 2})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 7.2 KB


In [7]:
# Importando o PyTorch
import torch
import torch.nn as nn

In [8]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Preparar e separar os dataframes
X = df.drop(['Id', 'Species'], axis = 1).values
y = df['Species'].values

# Separando os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transformando os dados em tensores
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [9]:
# Visualizando os dados de treino
X_train[0:5]

tensor([[4.6000, 3.6000, 1.0000, 0.2000],
        [5.7000, 4.4000, 1.5000, 0.4000],
        [6.7000, 3.1000, 4.4000, 1.4000],
        [4.8000, 3.4000, 1.6000, 0.2000],
        [4.4000, 3.2000, 1.3000, 0.2000]])

In [10]:
# Visualindos as resposta de treino
y_train[0:5]

tensor([1, 1, 0, 1, 1])

In [11]:
# Definindo uma rede neural (fully connected)
# Input layer: 4 entradas (variáveis de entrada) -> 16 saídas (arbitrário/definido por nós)
# Hideen layer: 16 entradas (combinando com as saídas da camada anterior) -> 12 saídas (arbitrário/definido por nós)
# Output layer: 12 entradas (combinando com as saídas da camada anterior) -> 3 saídas (são 3 espécies a serem previstas)
# Vamos usar ReLu como função de ativação

# Criando a rede neural usando nn.Sequential
model = nn.Sequential(nn.Linear(4, 16),
                      nn.ReLU(),
                      nn.Linear(16, 12),
                      nn.ReLU(),
                      nn.Linear(12, 3))

model

Sequential(
  (0): Linear(in_features=4, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=12, bias=True)
  (3): ReLU()
  (4): Linear(in_features=12, out_features=3, bias=True)
)

In [12]:
# Temos que definir a função de erro e o otimizador (que vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [13]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 100

# Erros
running_loss = []

# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Foward Propagation (passando os dados de treino pela rede)
    outputs = model.forward(X_train)
    # Calculando o erro
    loss = error_function(outputs, y_train)
    # Guadando o erro
    running_loss.append(loss)
    
    # Exibindo o erro de 10 em 10 épocas
    if i % 10 == 0:
        print(f'Epoch: {i} / Loss: {loss}')
        
    # Back Propagation
    # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
    optimizer.zero_grad()
    # Calcular os novos pesos
    loss.backward()
    # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
    optimizer.step()

Epoch: 0 / Loss: 1.1436221599578857
Epoch: 10 / Loss: 0.2684209942817688
Epoch: 20 / Loss: 0.14562295377254486
Epoch: 30 / Loss: 0.11020524054765701
Epoch: 40 / Loss: 0.06348980963230133
Epoch: 50 / Loss: 0.06151952967047691
Epoch: 60 / Loss: 0.05810494348406792
Epoch: 70 / Loss: 0.057134490460157394
Epoch: 80 / Loss: 0.055544860661029816
Epoch: 90 / Loss: 0.05461667850613594


In [14]:
# Previsões para os dados de teste

# Lista das previsões
preds = []

# Colocar a rede em modo de execução/previsão / tirar do modo de treinamento
with torch.no_grad():
    for val in X_test:
        predict = model.forward(val)
        preds.append(predict.argmax().item())
        
preds

[0,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1]

In [15]:
# Verificando os valores reais
y_test

tensor([0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 2, 0, 0, 2, 1, 2, 1, 2,
        2, 2, 2, 2, 1, 1])

In [16]:
# Vamos criar um Rede Neural no Pytorch usando uma subclasse de nn.Module

# Importando a biblioteca funcional do PyTorch
import torch.nn.functional as F

# Definindo a classe
class RedeNeural(nn.Module):
    
    # Função de inicialização da rede
    def __init__(self):
        # Chamada ao método __init__ da classe mãe
        super().__init__()
        # Vamos definir as camadas
        self.input = nn.Linear(4, 16)
        self.hidden = nn.Linear(16,12)
        self.output = nn.Linear(12, 3)
        
    # Função para executar o Feed Forward (Forward Propagation)
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = self.output(x)
        
        return x
    
model2 = RedeNeural()

model2

RedeNeural(
  (input): Linear(in_features=4, out_features=16, bias=True)
  (hidden): Linear(in_features=16, out_features=12, bias=True)
  (output): Linear(in_features=12, out_features=3, bias=True)
)

In [17]:
# Treinamento do modelo model2

# Redefinindo o optimizer
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)

# Definindo o número de épocas
epochs = 100

# Erros
running_loss = []

# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Foward Propagation (passando os dados de treino pela rede)
    outputs = model2.forward(X_train)
    # Calculando o erro
    loss = error_function(outputs, y_train)
    # Guadando o erro
    running_loss.append(loss)
    
    # Exibindo o erro de 10 em 10 épocas
    if i % 10 == 0:
        print(f'Epoch: {i} / Loss: {loss}')
        
    # Back Propagation
    # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
    optimizer.zero_grad()
    # Calcular os novos pesos
    loss.backward()
    # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
    optimizer.step()

Epoch: 0 / Loss: 1.1014240980148315
Epoch: 10 / Loss: 0.6796582341194153
Epoch: 20 / Loss: 0.4025101959705353
Epoch: 30 / Loss: 0.2126961052417755
Epoch: 40 / Loss: 0.10914194583892822
Epoch: 50 / Loss: 0.07543078064918518
Epoch: 60 / Loss: 0.06474893540143967
Epoch: 70 / Loss: 0.0608326755464077
Epoch: 80 / Loss: 0.058997463434934616
Epoch: 90 / Loss: 0.057861316949129105


In [18]:
# Previsões para os dados de teste usando o model2

# Lista das previsões
preds = []

# Colocar a rede em modo de execução/previsão / tirar do modo de treinamento
with torch.no_grad():
    for val in X_test:
        predict = model2.forward(val)
        preds.append(predict.argmax().item())
        
preds

[0,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1]

In [19]:
# Verificando os valores reais
y_test

tensor([0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 2, 0, 0, 2, 1, 2, 1, 2,
        2, 2, 2, 2, 1, 1])